In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [1]:
cd /media/datastorage/Phong/cifar10/cifar-10-batches-py/

/media/datastorage/Phong/cifar10/cifar-10-batches-py


In [ ]:
## fix for multi_gpu_model prediction time longer
from keras.layers import Lambda, concatenate
from keras import Model
import tensorflow as tf

def multi_gpu_model(model, gpus):
    if isinstance(gpus, (list, tuple)):
        num_gpus = len(gpus)
        target_gpu_ids = gpus
    else:
        num_gpus = gpus
        target_gpu_ids = range(num_gpus)

    def get_slice(data, i, parts):
        shape = tf.shape(data)
        batch_size = shape[:1]
        input_shape = shape[1:]
        step = batch_size // parts
        if i == num_gpus - 1:
            size = batch_size - step * i
        else:
            size = step
        size = tf.concat([size, input_shape], axis=0)
        stride = tf.concat([step, input_shape * 0], axis=0)
        start = stride * i
        return tf.slice(data, start, size)

    all_outputs = []
    for i in range(len(model.outputs)):
        all_outputs.append([])

    # Place a copy of the model on each GPU,
    # each getting a slice of the inputs.
    for i, gpu_id in enumerate(target_gpu_ids):
        with tf.device('/gpu:%d' % gpu_id):
            with tf.name_scope('replica_%d' % gpu_id):
                inputs = []
                # Retrieve a slice of the input.
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_i = Lambda(get_slice,
                                   output_shape=input_shape,
                                   arguments={'i': i,
                                              'parts': num_gpus})(x)
                    inputs.append(slice_i)

                # Apply model on slice
                # (creating a model replica on the target device).
                outputs = model(inputs)
                if not isinstance(outputs, list):
                    outputs = [outputs]

                # Save the outputs for merging back together later.
                for o in range(len(outputs)):
                    all_outputs[o].append(outputs[o])

    # Merge outputs on CPU.
    with tf.device('/cpu:0'):
        merged = []
        for name, outputs in zip(model.output_names, all_outputs):
            merged.append(concatenate(outputs,
                                    axis=0, name=name))
        return Model(model.inputs, merged)


In [ ]:
from keras.callbacks import Callback
import pickle
import sys

#Stop training on val_acc
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

#Save large model using pickle formate instead of h5            
class SaveCheckPoint(Callback):
    def __init__(self, model, dest_folder):
        super(Callback, self).__init__()
        self.model = model
        self.dest_folder = dest_folder
        
        #initiate
        self.best_val_acc = 0
        self.best_val_loss = sys.maxsize #get max value
          
    def on_epoch_end(self, epoch, logs={}):
        val_acc = logs['val_acc']
        val_loss = logs['val_loss']

        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            
            # Save weights in pickle format instead of h5
            print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
            weigh= self.model.get_weights()

            #now, use pickle to save your model weights, instead of .h5
            #for heavy model architectures, .h5 file is unsupported.
            fpkl= open(self.dest_folder, 'wb') #Python 3
            pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
            fpkl.close()
            
#             model.save('tmp.h5')
        elif val_acc == self.best_val_acc:
            if val_loss < self.best_val_loss:
                self.best_val_loss=val_loss
                
                # Save weights in pickle format instead of h5
                print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
                weigh= self.model.get_weights()

                #now, use pickle to save your model weights, instead of .h5
                #for heavy model architectures, .h5 file is unsupported.
                fpkl= open(self.dest_folder, 'wb') #Python 3
                pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
                fpkl.close()                    

In [ ]:
#501
#MUL 3 InceptionResNetV2
# from keras.applications import InceptionV3
# from keras.applications import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, GRU, Reshape, Concatenate,Bidirectional
from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

f3_base_1 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299,299,3))  
# for layer in f3_base.layers:
#     layer.trainable = False 
f3_x_1 = f3_base_1.output
f3_x_1 = GlobalAveragePooling2D()(f3_x_1)

f3_x_1 = Reshape([1,1536])(f3_x_1)  
f3_x_1 = GRU(2048, 
            return_sequences=False,                       
#             dropout=0.8                                     
            input_shape=[1,1536])(f3_x_1)

#Regularization with noise
f3_x_1 = GaussianNoise(0.1)(f3_x_1)

f3_x_1 = Dense(2048, activation='relu')(f3_x_1)
f3_x_1 = Dense(10, activation='softmax')(f3_x_1)
model_3_1 = Model(inputs=[f3_base_1.input],outputs=[f3_x_1])

print(model_3_1.summary())

In [ ]:
model_3_1.load_weights(os.path.join('checkpoints', 'Cifar10_IcpResNetV2_LRG299_Mul_GRU_L3_SGL.hdf5'))

In [ ]:
#501
#MUL 3 InceptionResNetV2
# from keras.applications import InceptionV3
# from keras.applications import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM
from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

f3_base_2 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299,299,3))  
# for layer in f3_base.layers:
#     layer.trainable = False 
f3_x_2 = f3_base_2.output
f3_x_2 = GlobalAveragePooling2D()(f3_x_2)

#Regularization with noise
f3_x_2 = GaussianNoise(0.1)(f3_x_2)

f3_x_2 = Dense(2048, activation='relu')(f3_x_2)
f3_x_2 = Dense(10, activation='softmax')(f3_x_2)
model_3_2 = Model(inputs=[f3_base_2.input],outputs=[f3_x_2])

print(model_3_2.summary())

In [ ]:
model_3_2.load_weights(os.path.join('checkpoints', 'Cifar10_IcpResNetV2_LRG299_Mul_STD_L2_SGL.hdf5'))

In [ ]:
#501
#MUL 3 InceptionResNetV2
# from keras.applications import InceptionV3
# from keras.applications import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, Reshape, Concatenate, Bidirectional
from keras.applications.inception_resnet_v2 import preprocess_input

from keras.layers import GaussianNoise

f3_base_3 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299,299,3))  
# for layer in f3_base.layers:
#     layer.trainable = False 
f3_x_3 = f3_base_3.output
f3_x_3 = GlobalAveragePooling2D()(f3_x_3)

#ADV
f3_x_3 = Reshape([1,1536])(f3_x_3)  
f3_x_3 = Bidirectional(LSTM(1024, 
                                 return_sequences=False, 
#                                  dropout=0.8
                                ),
                            input_shape=[1,1536],
                            merge_mode='concat')(f3_x_3)

#Regularization with noise
f3_x_3 = GaussianNoise(0.1)(f3_x_3)

f3_x_3 = Dense(2048, activation='relu')(f3_x_3)
f3_x_3 = Dense(10, activation='softmax')(f3_x_3)
model_3_3 = Model(inputs=[f3_base_3.input],outputs=[f3_x_3])

print(model_3_3.summary())

In [ ]:
model_3_3.load_weights(os.path.join('checkpoints', 'Cifar10_IcpResNetV2_LRG299_Mul_LSTM_L2_SGL.hdf5'))

In [ ]:
for i, layer in enumerate(model_3_1.layers):
    layer.name = layer.name + 'grp1_m1'

In [ ]:
for i, layer in enumerate(model_3_2.layers):
    layer.name = layer.name + 'grp1_m2'

In [ ]:
for i, layer in enumerate(model_3_3.layers):
    layer.name = layer.name + 'grp1_m3'

In [ ]:
# 3 models (2mul + 1 +1)
# 501
# pre-trainned weight
# get pretrained [-1] layer

# from keras.applications import InceptionV3
from keras.applications import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, LeakyReLU, Dropout
#from keras.applications.inception_v3 import preprocess_input
from keras_applications.imagenet_utils import preprocess_input

f1_mul_x = model_3_1.layers[-1].output 
f2_mul_x = model_3_2.layers[-1].output
f3_mul_x = model_3_3.layers[-1].output

x = concatenate([f1_mul_x, f2_mul_x, f3_mul_x])

x = Dense(4096)(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model_mul = Model(inputs=[model_3_1.get_input_at(0),model_3_2.get_input_at(0),model_3_3.get_input_at(0)],outputs=[x])

print(model_mul.summary())

In [ ]:
for layer in model_mul.layers[:-4]:
    layer.trainable=False
# for layer in model.layers[87:]:
#     layer.trainable=True)
for i,layer in enumerate(model_mul.layers):
    print(i,layer.name,layer.trainable)

In [ ]:
# generators for three inputs
def train_generator_three_img(X1, batch_size):
    gen = ImageDataGenerator(    
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,##
    #     brightness_range=[0.5, 1.5],##
        channel_shift_range=10,##
        fill_mode='nearest')
    
    genX1 = gen.flow_from_directory(X1, batch_size=batch_size, seed=1, target_size = (299, 299))    
    while True:
        X1i = genX1.next()
        yield [X1i[0], X1i[0], X1i[0]], X1i[1]
        
def valid_generator_three_img(X1, batch_size):
    gen = ImageDataGenerator()
    
    # shuffle = False
    genX1 = gen.flow_from_directory(X1, batch_size=batch_size, shuffle = False, target_size = (299, 299))        
    while True:
        X1i = genX1.next()
        yield [X1i[0], X1i[0], X1i[0]], X1i[1]    
    
def test_generator_three_img(X1, batch_size):
    gen = ImageDataGenerator(
#         preprocessing_function=preprocess_input
    )

    genX1 = gen.flow_from_directory(X1, batch_size=batch_size, shuffle=False, target_size = (299, 299))        
    while True:
        X1i_0, X1i_1 = next(genX1)
        yield [X1i_0, X1i_0, X1i_0]            

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

## fix for multi_gpu_model prediction time longer
from keras.layers import Lambda, concatenate
from keras import Model
import tensorflow as tf

NUM_GPU = 4
batch_size = 200
#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    gpu_model_mul = multi_gpu_model(model_mul, gpus=NUM_GPU)

epochs = 10##!!!
lr = 1e-4
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

if NUM_GPU != 1:
    gpu_model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

# model_txt = 'bi-lstm'
# # Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1.pkl')

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.99, verbose=1)
savecheckpoint = SaveCheckPoint(model_mul, savedfilename)

# # Helper: TensorBoard
# tb = TensorBoard(log_dir=os.path.join('cifar10_output', 'logs', model_txt))

# # Helper: Save results.
# timestamp = time.time()
# csv_logger = CSVLogger(os.path.join('cifar10_output', 'logs', model_txt + '-' + 'training-' + \
#     str(timestamp) + '.log'))

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = gpu_model_mul.fit_generator(
    generator = train_generator_three_img('train_resized_299',batch_size=batch_size), 
    steps_per_epoch = step_size_train,
    validation_data = valid_generator_three_img('test_resized_299',batch_size=batch_size),
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, savecheckpoint, earlystopping], 
    callbacks=[savecheckpoint, earlystopping], 
    verbose=1)    

In [ ]:
#use pickle to load model weights
import os
import pickle

pklfile= os.path.join('checkpoints', 'Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

# #use set_weights to load the modelweights into the model architecture
# NUM_GPU = 4
# #Using multiple models if more than 1 GPU
# if NUM_GPU != 1:
#     model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

gpu_model_mul.set_weights(weigh)

m3_sgl = gpu_model_mul.layers[-2]

#get weight from single gpu
weigh= m3_sgl.get_weights()

#now, use pickle to save model weights, instead of .h5
#for heavy model architectures, .h5 file is unsupported.
pklfile= os.path.join('checkpoints', 'Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1_SGL.pkl')

fpkl= open(pklfile, 'wb') #Python 3
pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
fpkl.close()

In [ ]:
#change test_generator
import numpy as np

# PREDICT ON OFFICIAL TEST
test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=gpu_model_mul.predict_generator(test_generator_three_img('test_resized_299',batch_size), steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1_SGL.csv')
results.head(10)

In [ ]:
mkdir pred_npy

In [ ]:
np.save(os.path.join('pred_npy','Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1.npy'), predict1)

In [ ]:
cp Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1_SGL.csv /home/bribeiro/Phong/Nat19/Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1_SGL.csv

In [ ]:
#SET 1

from classification_models.keras import Classifiers
from keras.models import Model
# from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, SimpleRNN, Reshape, Concatenate,Bidirectional
# from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

senet, preprocess_input = Classifiers.get('seresnext101')

f2_base = senet(input_shape=(299,299,3), weights='imagenet', include_top=False)
f2_x = GlobalAveragePooling2D()(f2_base.output)

#ADV Model
# f2_x = Reshape([1,2048])(f2_x)  
# f2_x = SimpleRNN(2048, 
#             return_sequences=False,                       
# #             dropout=0.8                                     
#             input_shape=[1,2048])(f2_x)

#Regularization with noise
f2_x = GaussianNoise(0.1)(f2_x)

f2_x = Dense(2048, activation='relu')(f2_x)
f2_x = Dense(10, activation='softmax')(f2_x)
model_2 = Model(inputs=[f2_base.input],outputs=[f2_x])

print(model_2.summary())
# # print(f2_base.summary())

In [ ]:
# model 2 - SeResNext101
#use pickle to load model weights
import os
import pickle

# load to multi gpu
NUM_GPU = 4
model_mul_2 = multi_gpu_model(model_2, gpus=NUM_GPU)

#load model the saved weights
pklfile= os.path.join('checkpoints', 'Cifar10_SEResNext101_LRG299_Mul_STD_L3.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

#set the saved weight to mul-gpus model
model_mul_2.set_weights(weigh)


In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import time, os
from math import ceil

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 40

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul_2.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cifar10_SeResNet101_MulModels299_STD_L3.csv')
results.head()

In [ ]:
np.save(os.path.join('pred_npy','Cifar10_SEResNext101_LRG299_L3.npy'), predict1)

In [ ]:
#Set 1
#MUL 1 - Xception

# from keras.applications import InceptionV3
from keras.applications import Xception
# from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, Reshape, Bidirectional
# from keras.applications.inception_v3 import preprocess_input
from keras.applications.xception import preprocess_input

from keras.layers import GaussianNoise

f1_base = Xception(weights='imagenet', include_top=False, input_shape=(299,299,3))  
f1_x = f1_base.output
f1_x = GlobalAveragePooling2D()(f1_x)

# f1_x = Reshape([1,2048])(f1_x)  
# f1_x = Bidirectional(LSTM(1024, 
#                                  return_sequences=False, 
# #                                  dropout=0.8
#                                 ),
#                             input_shape=[1,2048],
#                             merge_mode='concat')(f1_x)

#Regularization with noise
f1_x = GaussianNoise(0.1)(f1_x)

f1_x = Dense(2048, activation='relu')(f1_x)
f1_x = Dense(10, activation='softmax')(f1_x)
model_1 = Model(inputs=[f1_base.input],outputs=[f1_x])

print(model_1.summary())

In [ ]:
#Using multiple models if more than 1 GPU
NUM_GPU = 4
if NUM_GPU != 1:
    model_mul_1 = multi_gpu_model(model_1, gpus=NUM_GPU)

model_mul_1.load_weights(os.path.join('checkpoints', 'Cifar10_Xception_LRG299_Mul_STD_L2.hdf5'))

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import time, os
from math import ceil

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 40

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul_1.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cifar10_Xception_MulModels299_STD_L2.csv')
results.head()

In [ ]:
np.save(os.path.join('pred_npy','Cifar10_Xception_LRG299_L2.npy'), predict1)

In [ ]:
mean_pred3_1 = np.load(os.path.join('pred_npy','Cifar10_Xception_LRG299_L2.npy'))
mean_pred3_2 = np.load(os.path.join('pred_npy','Cifar10_SEResNext101_LRG299_L3.npy'))
mean_pred3_3 = np.load(os.path.join('pred_npy','Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1.npy'))
mean_pred3 = (mean_pred3_1+mean_pred3_2+mean_pred3_3)/3
print(mean_pred3[0:9])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np

predicted_class_indices_mean=np.argmax(mean_pred3,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1211_v1.csv')
results.head(20)

In [ ]:
cp Cifar10_MulModels299_AVG_Assembe_1211_v1.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1211_v1.csv

In [ ]:
import numpy as np
import os

mean_pred3_1 = np.load(os.path.join('pred_npy','Cifar10_Xception_MulModels299_GRU_L3.npy'))
mean_pred3_2 = np.load(os.path.join('pred_npy','Cifar10_SEResNext101_LRG299_L3.npy'))
mean_pred3_3 = np.load(os.path.join('pred_npy','Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1.npy'))
mean_pred3 = (mean_pred3_1+mean_pred3_2+mean_pred3_3)/3
print(mean_pred3[0:9])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np
from keras_applications.imagenet_utils import preprocess_input

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 40

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

predicted_class_indices_mean=np.argmax(mean_pred3,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1211_v2.csv')
results.head(20)

In [ ]:
cp Cifar10_MulModels299_AVG_Assembe_1211_v2.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1211_v2.csv

In [ ]:
a1 = mean_pred3_1
a2 = mean_pred3_2
a3 = mean_pred3_3

In [ ]:
# print(a1)
print(mean_pred3_1)

In [ ]:
np.argmax(a1[0], axis=0) 

In [ ]:
idx_max = np.argmax(a1[0], axis=0) 
for idx in range(10):
    if (idx != idx_max):
        a1[0][idx] = 0
    else:
        a1[0][idx] = 1

print(a1[0])        

In [ ]:
for i in range(len(a1)):
    i_max = np.argmax(a1[i], axis=0)
    for idx in range(10):
        if idx != i_max:
            a1[i][idx] = 0
        else:
            a1[i][idx] = 1            
print(a1)            

In [ ]:
for i in range(len(a2)):
    i_max = np.argmax(a2[i], axis=0)
    for idx in range(10):
        if idx != i_max:
            a2[i][idx] = 0
        else:
            a2[i][idx] = 1             
print(a2)  

In [ ]:
for i in range(len(a3)):
    i_max = np.argmax(a3[i], axis=0)
    for idx in range(10):
        if idx != i_max:
            a3[i][idx] = 0
        else:
            a3[i][idx] = 1             
print(a3)  

In [ ]:
mean_a = (a1+a2+a3)/3

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np
from keras_applications.imagenet_utils import preprocess_input

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 40

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

predicted_class_indices_mean=np.argmax(mean_a,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1211_v4.csv')
results.head(20)

In [ ]:
cp Cifar10_MulModels299_AVG_Assembe_1211_v4.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1211_v4.csv

In [ ]:
idx_max = np.argmax(a1[0], axis=0) 
max_val = a1[0][idx_max]
a1[0][idx_max] = 0
idx_max2 = np.argmax(a1[0], axis=0) 

for idx in range(10):
    if (idx != idx_max2):
        a1[0][idx] = 0

a1[0][idx_max] = max_val

print(a1[0])        

In [ ]:
print(a1[0])

In [ ]:
for i in range(len(a1)):
    idx_max = np.argmax(a1[i], axis=0) 
    max_val = a1[i][idx_max]
    a1[i][idx_max] = 0
    idx_max2 = np.argmax(a1[i], axis=0) 

    for idx in range(10):
        if (idx != idx_max2):
            a1[i][idx] = 0

    a1[i][idx_max] = max_val

print(a1[0:50])  

In [11]:
for i in range(len(a2)):
    idx_max = np.argmax(a2[i], axis=0) 
    max_val = a2[i][idx_max]
    a2[i][idx_max] = 0
    idx_max2 = np.argmax(a2[i], axis=0) 

    for idx in range(10):
        if (idx != idx_max2):
            a2[i][idx] = 0

    a2[i][idx_max] = max_val

print(a2[0:10]) 

[[9.9999321e-01 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 6.2717859e-06 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 5.3751616e-09 0.0000000e+00]
 [9.9999833e-01 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 1.6641090e-06 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00 5.6560179e-10 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 6.5428336e-11 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00 1.0123520e-10 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00 1.2328574e-09 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 

In [ ]:
for i in range(len(a3)):
    idx_max = np.argmax(a3[i], axis=0) 
    max_val = a3[i][idx_max]
    a3[i][idx_max] = 0
    idx_max2 = np.argmax(a3[i], axis=0) 

    for idx in range(10):
        if (idx != idx_max2):
            a3[i][idx] = 0

    a3[i][idx_max] = max_val

print(a3[0:10]) 

In [ ]:
mean_a =(a1+a2+a3)/3
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np
from keras_applications.imagenet_utils import preprocess_input

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 40

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

predicted_class_indices_mean=np.argmax(mean_a,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1211_v5.csv')
results.head(20)

In [ ]:
cp Cifar10_MulModels299_AVG_Assembe_1211_v5.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1211_v5.csv

In [ ]:
cp /home/bribeiro/Phong/Nat19/Cifar10_Eff_LRGB4224_Mul.npy pred_npy/Cifar10_Eff_LRGB4224_Mul.npy

In [ ]:
import numpy as np
import os

mean_pred3_4 = np.load(os.path.join('pred_npy','Cifar10_Eff_LRGB4224_Mul.npy'))

In [ ]:
a4 = mean_pred3_4

In [ ]:
for i in range(len(a4)):
    idx_max = np.argmax(a4[i], axis=0) 
    max_val = a4[i][idx_max]
    a4[i][idx_max] = 0
    idx_max4 = np.argmax(a4[i], axis=0) 

    for idx in range(10):
        if (idx != idx_max4):
            a4[i][idx] = 0

    a4[i][idx_max] = max_val

print(a4[0:10]) 

In [ ]:
mean_a = (a2+a3+a4)/3

In [ ]:

from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np
from keras_applications.imagenet_utils import preprocess_input

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 30

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

predicted_class_indices_mean=np.argmax(mean_a,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1411_v1.csv')
results.head(20)

In [ ]:
cp Cifar10_MulModels299_AVG_Assembe_1411_v1.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1411_v1.csv

In [ ]:
mean_a = (a1+a2+a3+a4)/4

from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np
from keras_applications.imagenet_utils import preprocess_input

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 30

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

predicted_class_indices_mean=np.argmax(mean_a,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1411_v3.csv')
results.head(20)

In [ ]:
cp Cifar10_MulModels299_AVG_Assembe_1411_v3.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1411_v3.csv

In [2]:
import numpy as np
import os

# mean_pred3_1 = np.load(os.path.join('pred_npy','Cifar10_Xception_MulModels299_GRU_L3.npy'))
mean_pred3_2 = np.load(os.path.join('pred_npy','Cifar10_SEResNext101_LRG299_L3.npy'))
mean_pred3_3 = np.load(os.path.join('pred_npy','Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1.npy'))
mean_pred3_5 = np.load(os.path.join('pred_npy','Cifar10_Eff_B5_345_L2.npy'))

mean_pred3 = (mean_pred3_5+mean_pred3_2+mean_pred3_3)/3
print(mean_pred3[0:9])

[[9.98876572e-01 1.06719475e-04 1.47624305e-04 1.06542582e-04
  1.17790914e-04 1.20928911e-04 1.17569034e-04 1.05496845e-04
  1.97017565e-04 1.03772152e-04]
 [9.98881161e-01 1.06688065e-04 1.45969389e-04 1.06549182e-04
  1.17798169e-04 1.20936718e-04 1.17576616e-04 1.05503765e-04
  1.94046515e-04 1.03778664e-04]
 [9.98880625e-01 1.06690895e-04 1.45967075e-04 1.06549356e-04
  1.17796997e-04 1.20935350e-04 1.17575102e-04 1.05502455e-04
  1.94600216e-04 1.03781982e-04]
 [9.98881280e-01 1.06680782e-04 1.45959479e-04 1.06542291e-04
  1.17790558e-04 1.20928620e-04 1.17568685e-04 1.05496641e-04
  1.94018692e-04 1.03771563e-04]
 [9.98881161e-01 1.06680985e-04 1.45961953e-04 1.06576830e-04
  1.17790732e-04 1.20928766e-04 1.17568874e-04 1.05496751e-04
  1.94019216e-04 1.03771767e-04]
 [9.98881280e-01 1.06680985e-04 1.45959537e-04 1.06542408e-04
  1.17790674e-04 1.20928737e-04 1.17568852e-04 1.05496751e-04
  1.94018954e-04 1.03771767e-04]
 [9.98881280e-01 1.06680913e-04 1.45959755e-04 1.06542291e

In [3]:
# mean_a = (a1+a2+a3+a4)/4

from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np
from keras_applications.imagenet_utils import preprocess_input

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 30

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

predicted_class_indices_mean=np.argmax(mean_pred3,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1511_v1.csv')
results.head(20)

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarn

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


,id,predicted
0,0/aeroplane_s_000002.png,0
1,0/aeroplane_s_000040.png,0
2,0/aeroplane_s_000045.png,0
3,0/aeroplane_s_000063.png,0
4,0/airbus_s_000009.png,0
5,0/airbus_s_000030.png,0
6,0/airbus_s_000049.png,0
7,0/airbus_s_000099.png,0
8,0/airbus_s_000108.png,0
9,0/airbus_s_000131.png,0


In [6]:
cp Cifar10_MulModels299_AVG_Assembe_1511_v1.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1511_v1.csv

In [14]:
a5 = mean_pred3_5
a2 = mean_pred3_2
a3 = mean_pred3_3

In [12]:
for i in range(len(a2)):
#     idx_max = np.argmax(a2[i], axis=0) 
#     max_val = a2[i][idx_max]
#     a2[i][idx_max] = 0
    idx_max2 = np.argmax(a2[i], axis=0) 

    for idx in range(10):
        if (idx != idx_max2):
            a2[i][idx] = 0

#     a2[i][idx_max] = max_val

print(a2[0:2]) 

[[0.9999932 0.        0.        0.        0.        0.        0.
  0.        0.        0.       ]
 [1.        0.        0.        0.        0.        0.        0.
  0.        0.        0.       ]]


In [13]:
for i in range(len(a3)):
#     idx_max = np.argmax(a3[i], axis=0) 
#     max_val = a3[i][idx_max]
#     a3[i][idx_max] = 0
    idx_max3 = np.argmax(a3[i], axis=0) 

    for idx in range(10):
        if (idx != idx_max3):
            a3[i][idx] = 0

#     a3[i][idx_max] = max_val

print(a3[0:2]) 

[[0.9966438  0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.99664354 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]]


In [15]:
for i in range(len(a5)):
#     idx_max = np.argmax(a3[i], axis=0) 
#     max_val = a3[i][idx_max]
#     a3[i][idx_max] = 0
    idx_max5 = np.argmax(a5[i], axis=0) 

    for idx in range(10):
        if (idx != idx_max5):
            a5[i][idx] = 0

#     a3[i][idx_max] = max_val

print(a5[0:2])

[[0.9999927 0.        0.        0.        0.        0.        0.
  0.        0.        0.       ]
 [1.        0.        0.        0.        0.        0.        0.
  0.        0.        0.       ]]


In [16]:
mean_a = (a5+a2+a3)/3

from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np
from keras_applications.imagenet_utils import preprocess_input

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 30

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

predicted_class_indices_mean=np.argmax(mean_a,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1511_v2.csv')
results.head(20)

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


,id,predicted
0,0/aeroplane_s_000002.png,0
1,0/aeroplane_s_000040.png,0
2,0/aeroplane_s_000045.png,0
3,0/aeroplane_s_000063.png,0
4,0/airbus_s_000009.png,0
5,0/airbus_s_000030.png,0
6,0/airbus_s_000049.png,0
7,0/airbus_s_000099.png,0
8,0/airbus_s_000108.png,0
9,0/airbus_s_000131.png,0


In [17]:
cp Cifar10_MulModels299_AVG_Assembe_1511_v2.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1511_v2.csv

In [18]:
import numpy as np
import os

# mean_pred3_1 = np.load(os.path.join('pred_npy','Cifar10_Xception_MulModels299_GRU_L3.npy'))
mean_pred3_2 = np.load(os.path.join('pred_npy','Cifar10_SEResNext101_LRG299_L3.npy'))
mean_pred3_3 = np.load(os.path.join('pred_npy','Cifar10_LRG299_Mul_Pretrained_V3_3MIcpRes_STD_L1.npy'))
mean_pred3_5_2 = np.load(os.path.join('pred_npy','Cifar10_Eff_B5_345_L2_TTA3.npy'))

mean_pred3 = (mean_pred3_5_2+mean_pred3_2+mean_pred3_3)/3
print(mean_pred3[0:2])

[[9.98878717e-01 1.06719403e-04 1.46113845e-04 1.06542517e-04
  1.17790849e-04 1.20928911e-04 1.17569019e-04 1.05496838e-04
  1.96383786e-04 1.03771956e-04]
 [9.98881161e-01 1.06688065e-04 1.45969374e-04 1.06549182e-04
  1.17798169e-04 1.20936718e-04 1.17576616e-04 1.05503765e-04
  1.94046515e-04 1.03778664e-04]]


In [19]:
# mean_a = (a5+a2+a3)/3

from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np
from keras_applications.imagenet_utils import preprocess_input

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 30

train_set = train_datagen.flow_from_directory('train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

predicted_class_indices_mean=np.argmax(mean_pred3,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=test_set1.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cifar10_MulModels299_AVG_Assembe_1511_v4.csv')
results.head(5)

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


,id,predicted
0,0/aeroplane_s_000002.png,0
1,0/aeroplane_s_000040.png,0
2,0/aeroplane_s_000045.png,0
3,0/aeroplane_s_000063.png,0
4,0/airbus_s_000009.png,0


In [20]:
cp Cifar10_MulModels299_AVG_Assembe_1511_v4.csv /home/bribeiro/Phong/Nat19/Cifar10_MulModels299_AVG_Assembe_1511_v4.csv

In [4]:
!pip check numpy

tensorflow 1.12.0 has requirement keras-applications>=1.0.6, but you have keras-applications 1.0.2.
tensorflow 1.12.0 has requirement keras-preprocessing>=1.0.5, but you have keras-preprocessing 1.0.1.
tensorflow 1.12.0 has requirement tensorboard<1.13.0,>=1.12.0, but you have tensorboard 1.8.0.
You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
!pip check Keras-Applications

tensorflow 1.12.0 has requirement keras-applications>=1.0.6, but you have keras-applications 1.0.2.
tensorflow 1.12.0 has requirement keras-preprocessing>=1.0.5, but you have keras-preprocessing 1.0.1.
tensorflow 1.12.0 has requirement tensorboard<1.13.0,>=1.12.0, but you have tensorboard 1.8.0.
You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import keras; print(keras.__version__)

In [ ]:
import sys
print(sys.version)

In [21]:
import tensorflow as tf; print(tf.__version__)

1.12.0


In [ ]:
import numpy
numpy.version.version

In [ ]:
!nvcc --version

In [22]:
!pip show tensorflow-gpu

Name: tensorflow-gpu
Version: 1.8.0
Summary: TensorFlow helps the tensors flow
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python2.7/dist-packages
Requires: astor, protobuf, gast, tensorboard, six, wheel, absl-py, backports.weakref, termcolor, enum34, numpy, grpcio, mock
Required-by: 
You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
!pip show jupyter